In [3]:
import geopandas as gpd
from shapely.geometry import Point

# Cesty k datům
linie_path = "czech_linie_trimmed.gpkg"
centroidy_path = "../data/czech/czech_regions_points.gpkg"
output_path = "czech_linie_oriented.gpkg"

# Načti data
gdf_lines = gpd.read_file(linie_path)
gdf_points = gpd.read_file(centroidy_path)

# Zajisti stejný CRS
gdf_points = gdf_points.to_crs(gdf_lines.crs)

# Udělej lookup: jméno regionu → Point
region_to_point = dict(zip(gdf_points["nazev"], gdf_points.geometry))

# Funkce na zjištění orientace podle blízkosti k centroidům
def find_orientation(row, tolerance=100):
    origin = row["origin"]
    dest = row["destination"]
    start = Point(row.geometry.coords[0])
    end = Point(row.geometry.coords[-1])
    
    origin_pt = region_to_point.get(origin)
    dest_pt = region_to_point.get(dest)

    if origin_pt and start.distance(origin_pt) < tolerance:
        return 1  # začíná v originu – vše OK
    elif origin_pt and end.distance(origin_pt) < tolerance:
        return -1  # začíná v destination → musíme přetočit
    else:
        return 0  # nepovedlo se spárovat (může být chyba v názvech)

# Aplikuj
gdf_lines["orientation"] = gdf_lines.apply(find_orientation, axis=1)

# Ulož výstup
gdf_lines.to_file(output_path, driver="GPKG", layer="linie_oriented")

print("✅ Hotovo – orientace určena podle originu vs. začátek linie.")


✅ Hotovo – orientace určena podle originu vs. začátek linie.
